In [1]:
import glob 
import csv
import pandas as pd
import numpy as np
from scipy import stats 

import plotly.graph_objects as go
import plotly.express as px

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [2]:
merged_csvs = glob.glob('../raw_data_files_clean/merged_*')
merged_csvs.sort()

merged_csvs

['../raw_data_files_clean/merged_assessment.csv',
 '../raw_data_files_clean/merged_assessment_2022.csv',
 '../raw_data_files_clean/merged_demographics.csv',
 '../raw_data_files_clean/merged_directory.csv']

In [3]:
def pull_files_and_create_df(raw_file_names): 
    df_list=[]
    for i in raw_file_names:
        name = i.split('/')[2].split('.')[0]
        globals()[f'df_{name}'] = pd.read_csv(i, low_memory=False)
        d = df_list.append(f'df_{name}')
    df_list.sort(reverse=True)
    return df_list

In [4]:
pull_files_and_create_df(merged_csvs)

['df_merged_directory',
 'df_merged_demographics',
 'df_merged_assessment_2022',
 'df_merged_assessment']

In [5]:
df_merged_assessment_2022['subpopulation'].unique()

array(['All SubPopulation', 'Students with Limited English Proficiency',
       'Homeless', 'Migrant', 'Students with Disabilities',
       'General Education Students', 'Economically Disadvantaged',
       'Foster', 'Military Family'], dtype=object)

In [7]:
df_merged_assessment_2022

,system_code,system_name_x,school_code,school_name_x,subject,grade,gender,race,ethnicity,subpopulation,first_year_el_exemption_percent,alternate_assessed_percent,percent_proficient,percent_level1,percent_level2,percent_level3,percent_level4,participation_rate,system_name_y,school_name_y,site_type,nces_id,is_charter,street,city,state,zip_5,county,file_year,opened_date,updated_district,updated_district_code,is_charter_district
0,0,Alabama State Department of Education,0,Alabama State Department of Education,Math,ALL,All Gender,All Race,All Ethnicity,All SubPopulation,0.00,1.26,27.23,30.50,42.26,20.07,7.16,98.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alabama State Department of Education,NaN,False
1,0,Alabama State Department of Education,0,Alabama State Department of Education,Math,ALL,All Gender,All Race,Hispanic/Latino,All SubPopulation,0.00,1.06,18.40,36.92,44.68,14.82,3.58,98.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alabama State Department of Education,NaN,False
2,0,Alabama State Department of Education,0,Alabama State Department of Education,Math,ALL,All Gender,All Race,Other Ethnicity,All SubPopulation,0.00,1.28,28.25,29.77,41.99,20.68,7.57,98.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alabama State Department of Education,NaN,False
3,0,Alabama State Department of Education,0,Alabama State Department of Education,Math,ALL,All Gender,Asian,All Ethnicity,All SubPopulation,0.00,1.12,62.23,9.56,28.21,32.35,29.89,98.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alabama State Department of Education,NaN,False
4,0,Alabama State Department of Education,0,Alabama State Department of Education,Math,ALL,All Gender,Asian,Other Ethnicity,All SubPopulation,0.00,1.14,63.40,8.94,27.67,32.83,30.56,98.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alabama State Department of Education,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053803,810,Magic City Acceptance Academy,10,Magic City Acceptance Academy,ELA,ALL,Female,White,All Ethnicity,Military Family,*,*,*,*,*,*,*,*,Magic City Acceptance Academy,Magic City Acceptance Academy,Public Charter School Startup,1.002110e+10,True,75 Bagby Dr,Birmingham,AL,35209-,Jefferson,2022.0,7/1/21,Homewood City,157.0,True
2053804,810,Magic City Acceptance Academy,10,Magic City Acceptance Academy,ELA,ALL,Female,White,Hispanic/Latino,Military Family,*,*,*,*,*,*,*,*,Magic City Acceptance Academy,Magic City Acceptance Academy,Public Charter School Startup,1.002110e+10,True,75 Bagby Dr,Birmingham,AL,35209-,Jefferson,2022.0,7/1/21,Homewood City,157.0,True
2053805,810,Magic City Acceptance Academy,10,Magic City Acceptance Academy,ELA,ALL,Female,White,Hispanic/Latino,Military Family,*,*,*,*,*,*,*,*,Magic City Acceptance Academy,Magic City Acceptance Academy,Public Charter School Startup,1.002110e+10,True,75 Bagby Dr,Birmingham,AL,35209-,Jefferson,2022.0,7/1/21,Homewood City,157.0,True
2053806,810,Magic City Acceptance Academy,10,Magic City Acceptance Academy,ELA,ALL,Female,White,Other Ethnicity,Military Family,*,*,*,*,*,*,*,*,Magic City Acceptance Academy,Magic City Acceptance Academy,Public Charter School Startup,1.002110e+10,True,75 Bagby Dr,Birmingham,AL,35209-,Jefferson,2022.0,7/1/21,Homewood City,157.0,True


In [6]:
df_subpops = df_merged_assessment_2022.loc[
    ((df_merged_assessment_2022['system_code']==0) | 
     ((df_merged_assessment_2022['is_charter_district']==True) 
      & (df_merged_assessment_2022['is_charter']==True)
      & (df_merged_assessment_2022['school_code']!=0)) | 
     ((df_merged_assessment_2022['is_charter_district']==True) 
      & (df_merged_assessment_2022['school_code']==0))) & 
                             (df_merged_assessment_2022['gender']=='All Gender') & 
                             (df_merged_assessment_2022['grade']=='ALL') & 
                             (df_merged_assessment_2022['ethnicity']=='All Ethnicity') & 
                             (df_merged_assessment_2022['race']=='All Race') & 
                             (df_merged_assessment_2022['subpopulation']!='All SubPopulation')  & 
                             (df_merged_assessment_2022['subpopulation']!='Homeless') &  
                             (df_merged_assessment_2022['subpopulation']!='Military Family') & 
                             (df_merged_assessment_2022['subpopulation']!='Migrant') & 
                             (df_merged_assessment_2022['subpopulation']!='Foster') & 
                             (df_merged_assessment_2022['subpopulation']!='General Education Students') 
    # & (df_merged_assessment_2022['subpopulation']=='Students with Limited English Proficiency')
].drop_duplicates()

df_subpops['equity_group'] = df_subpops['subpopulation']

df_subpops 

,system_code,system_name_x,school_code,school_name_x,subject,grade,gender,race,ethnicity,subpopulation,first_year_el_exemption_percent,alternate_assessed_percent,percent_proficient,percent_level1,percent_level2,percent_level3,percent_level4,participation_rate,system_name_y,school_name_y,site_type,nces_id,is_charter,street,city,state,zip_5,county,file_year,opened_date,updated_district,updated_district_code,is_charter_district,equity_group
1090,0,Alabama State Department of Education,0,Alabama State Department of Education,Math,ALL,All Gender,All Race,All Ethnicity,Students with Limited English Proficiency,0.00,0.85,10.35,49.45,40.19,8.66,1.69,98.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alabama State Department of Education,NaN,False,Students with Limited English Proficiency
1951,0,Alabama State Department of Education,0,Alabama State Department of Education,Math,ALL,All Gender,All Race,All Ethnicity,Students with Disabilities,0.00,9.08,8.12,61.07,30.82,6.73,1.39,97.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alabama State Department of Education,NaN,False,Students with Disabilities
2991,0,Alabama State Department of Education,0,Alabama State Department of Education,Math,ALL,All Gender,All Race,All Ethnicity,Economically Disadvantaged,0.00,1.44,15.11,41.39,43.50,12.58,2.53,97.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alabama State Department of Education,NaN,False,Economically Disadvantaged
400963,49,Mobile County,0,Mobile County,Math,ALL,All Gender,All Race,All Ethnicity,Students with Limited English Proficiency,0.00,0.81,10.93,50.23,38.84,9.88,1.05,98.06,Mobile County,Mobile County,Central Office,1.023700e+05,NaN,1 Magnum Pass,Mobile,AL,36618-0069,Mobile,2022.0,1/1/01,Mobile County,NaN,True,Students with Limited English Proficiency
401612,49,Mobile County,0,Mobile County,Math,ALL,All Gender,All Race,All Ethnicity,Students with Disabilities,0.00,9.59,6.23,64.14,29.62,5.22,1.01,97.67,Mobile County,Mobile County,Central Office,1.023700e+05,NaN,1 Magnum Pass,Mobile,AL,36618-0069,Mobile,2022.0,1/1/01,Mobile County,NaN,True,Students with Disabilities
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2050950,805,i3 Academy,15,i3 Academy - Middle School,ELA,ALL,All Gender,All Race,All Ethnicity,Economically Disadvantaged,1.69,0.00,18.97,32.76,48.28,17.24,1.72,98.33,i3 Academy,i3 Academy - Middle School,Public Charter School Startup,1.002040e+10,True,7901 1st Avenue North,Birmingham,AL,35206-,Jefferson,2022.0,7/1/21,Birmingham City,114.0,True,Economically Disadvantaged
2051669,808,Breakthrough Charter School,10,Breakthrough Charter School,ELA,ALL,All Gender,All Race,All Ethnicity,Students with Disabilities,*,*,*,*,*,*,*,*,Breakthrough Charter School,Breakthrough Charter School,Public Charter School Startup,1.002080e+10,True,1101 Washington Street,Marion,AL,36756-,Perry,2022.0,7/1/21,Perry County,53.0,True,Students with Disabilities
2051897,808,Breakthrough Charter School,10,Breakthrough Charter School,ELA,ALL,All Gender,All Race,All Ethnicity,Economically Disadvantaged,0.00,5.13,33.33,17.95,48.72,30.77,2.56,99.39,Breakthrough Charter School,Breakthrough Charter School,Public Charter School Startup,1.002080e+10,True,1101 Washington Street,Marion,AL,36756-,Perry,2022.0,7/1/21,Perry County,53.0,True,Economically Disadvantaged
2053106,810,Magic City Acceptance Academy,10,Magic City Acceptance Academy,ELA,ALL,All Gender,All Race,All Ethnicity,Students with Disabilities,0.00,0.00,54.55,18.18,27.27,45.45,9.09,91.67,Magic City Acceptance Academy,Magic City Acceptance Academy,Public Charter School Startup,1.002110e+10,True,75 Bagby Dr,Birmingham,AL,35209-,Jefferson,2022.0,7/1/21,Homewood City,157.0,True,Students with Disabilities


In [160]:
df_black = df_merged_assessment_2022.loc[
    ((df_merged_assessment_2022['system_code']==0) | 
     ((df_merged_assessment_2022['is_charter_district']==True) 
      & (df_merged_assessment_2022['is_charter']==True)
      & (df_merged_assessment_2022['school_code']!=0)) | 
     ((df_merged_assessment_2022['is_charter_district']==True) 
      & (df_merged_assessment_2022['school_code']==0))) & 
                             (df_merged_assessment_2022['gender']=='All Gender') & 
                             (df_merged_assessment_2022['grade']=='ALL') & 
                             (df_merged_assessment_2022['race']=='Black or African American') & 
                             (df_merged_assessment_2022['ethnicity']=='All Ethnicity') & 
                             (df_merged_assessment_2022['subpopulation']=='All SubPopulation') 
  
].drop_duplicates()


df_black['equity_group'] = df_black['race']

df_black

,system_code,system_name_x,school_code,school_name_x,subject,grade,gender,race,ethnicity,subpopulation,first_year_el_exemption_percent,alternate_assessed_percent,percent_proficient,percent_level1,percent_level2,percent_level3,percent_level4,participation_rate,system_name_y,school_name_y,site_type,nces_id,is_charter,street,city,state,zip_5,county,file_year,opened_date,updated_district,updated_district_code,is_charter_district,equity_group
5,0,Alabama State Department of Education,0,Alabama State Department of Education,Math,ALL,All Gender,Black or African American,All Ethnicity,All SubPopulation,0.00,1.60,10.78,48.45,40.77,9.28,1.50,97.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alabama State Department of Education,NaN,False,Black or African American
400050,49,Mobile County,0,Mobile County,Math,ALL,All Gender,Black or African American,All Ethnicity,All SubPopulation,0.00,1.76,11.27,45.78,42.95,9.87,1.40,97.47,Mobile County,Mobile County,Central Office,1.023700e+05,NaN,1 Magnum Pass,Mobile,AL,36618-0069,Mobile,2022.0,1/1/01,Mobile County,NaN,True,Black or African American
465917,51,Montgomery County,0,Montgomery County,Math,ALL,All Gender,Black or African American,All Ethnicity,All SubPopulation,0.00,1.40,6.39,57.82,35.79,5.66,0.74,95.96,Montgomery County,Montgomery County,Central Office,1.024300e+05,NaN,307 S Decatur St,Montgomery,AL,36104-4310,Montgomery,2022.0,1/1/01,Montgomery County,NaN,True,Black or African American
478759,51,Montgomery County,220,Davis Elementary School,Math,ALL,All Gender,Black or African American,All Ethnicity,All SubPopulation,0.00,0.00,2.80,77.57,19.63,2.34,0.47,95.11,Montgomery County,Davis Elementary School,Public Charter School - Converted,1.024300e+10,True,3605 Rosa Parks Dr,Montgomery,AL,36105-2099,Montgomery,2022.0,1/1/01,Montgomery County,51.0,True,Black or African American
507074,53,Perry County,0,Perry County,Math,ALL,All Gender,Black or African American,All Ethnicity,All SubPopulation,0.00,1.35,3.28,66.47,30.25,3.28,0.00,95.03,Perry County,Perry County,Central Office,1.026700e+05,NaN,200 Monroe St,Marion,AL,36756,Perry,2022.0,NaN,Perry County,NaN,True,Black or African American
562738,60,Sumter County,0,Sumter County,Math,ALL,All Gender,Black or African American,All Ethnicity,All SubPopulation,0.00,1.71,1.71,71.86,26.44,1.71,0.00,95.35,Sumter County,Sumter County,Central Office,1.030900e+05,NaN,Hwy 28 To Country Club Dr,Livingston,AL,35470,Sumter,2022.0,NaN,Sumter County,NaN,True,Black or African American
668354,114,Birmingham City,0,Birmingham City,Math,ALL,All Gender,Black or African American,All Ethnicity,All SubPopulation,0.00,1.12,8.06,61.06,30.88,6.87,1.19,96.43,Birmingham City,Birmingham City,Central Office,1.003900e+05,NaN,2015 Park Place North,Birmingham,AL,35203-0007,Jefferson,2022.0,1/1/01,Birmingham City,NaN,True,Black or African American
810324,157,Homewood City,0,Homewood City,Math,ALL,All Gender,Black or African American,All Ethnicity,All SubPopulation,0.00,3.29,32.34,28.44,39.22,26.65,5.69,98.97,Homewood City,Homewood City,Central Office,1.017600e+05,NaN,450 Dale Avenue,Homewood,AL,35209-3401,Jefferson,2022.0,NaN,Homewood City,NaN,True,Black or African American
1016251,800,Acceleration Day and Evening Academy,10,Acceleration Day and Evening Academy,Math,ALL,All Gender,Black or African American,All Ethnicity,All SubPopulation,0.00,0.00,2.34,57.03,40.63,2.34,0.00,100.00,Acceleration Day and Evening Academy,Acceleration Day and Evening Academy,Public Charter School Startup,1.001970e+10,True,3725 AIRPORT BLVDSuite 161,MOBILE,AL,36608,Mobile,2022.0,6/1/17,Mobile County,49.0,True,Black or African American
1018269,801,University Charter School,10,University Charter School,Math,ALL,All Gender,Black or African American,All Ethnicity,All SubPopulation,0.00,3.23,12.26,38.71,49.03,10.97,1.29,99.71,University Charter School,University Charter School,Public Charter School Startup,1.001990e+10,True,108 North Street,Livingston,AL,35470,Sumter,2022.0,6/19/18,Sumter County,60.0,True,Bla

In [161]:
df_hispanic = df_merged_assessment_2022.loc[
    ((df_merged_assessment_2022['system_code']==0) | 
     ((df_merged_assessment_2022['is_charter_district']==True) 
      & (df_merged_assessment_2022['is_charter']==True)
      & (df_merged_assessment_2022['school_code']!=0)) | 
     ((df_merged_assessment_2022['is_charter_district']==True) 
      & (df_merged_assessment_2022['school_code']==0))) & 
                             (df_merged_assessment_2022['gender']=='All Gender') & 
                             (df_merged_assessment_2022['grade']=='ALL') & 
                             (df_merged_assessment_2022['race']=='All Race') & 
                             (df_merged_assessment_2022['ethnicity']=='Hispanic/Latino') & 
                             (df_merged_assessment_2022['subpopulation']=='All SubPopulation') 
  
].drop_duplicates()


df_hispanic['equity_group'] = df_hispanic['ethnicity']

df_hispanic

,system_code,system_name_x,school_code,school_name_x,subject,grade,gender,race,ethnicity,subpopulation,first_year_el_exemption_percent,alternate_assessed_percent,percent_proficient,percent_level1,percent_level2,percent_level3,percent_level4,participation_rate,system_name_y,school_name_y,site_type,nces_id,is_charter,street,city,state,zip_5,county,file_year,opened_date,updated_district,updated_district_code,is_charter_district,equity_group
1,0,Alabama State Department of Education,0,Alabama State Department of Education,Math,ALL,All Gender,All Race,Hispanic/Latino,All SubPopulation,0.00,1.06,18.40,36.92,44.68,14.82,3.58,98.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alabama State Department of Education,NaN,False,Hispanic/Latino
400045,49,Mobile County,0,Mobile County,Math,ALL,All Gender,All Race,Hispanic/Latino,All SubPopulation,0.00,1.23,20.44,38.13,41.43,16.67,3.77,98.06,Mobile County,Mobile County,Central Office,1.023700e+05,NaN,1 Magnum Pass,Mobile,AL,36618-0069,Mobile,2022.0,1/1/01,Mobile County,NaN,True,Hispanic/Latino
465918,51,Montgomery County,0,Montgomery County,Math,ALL,All Gender,All Race,Hispanic/Latino,All SubPopulation,0.00,0.79,6.19,57.30,36.51,5.24,0.95,95.69,Montgomery County,Montgomery County,Central Office,1.024300e+05,NaN,307 S Decatur St,Montgomery,AL,36104-4310,Montgomery,2022.0,1/1/01,Montgomery County,NaN,True,Hispanic/Latino
478757,51,Montgomery County,220,Davis Elementary School,Math,ALL,All Gender,All Race,Hispanic/Latino,All SubPopulation,0.00,0.00,5.00,75.00,20.00,5.00,0.00,100.00,Montgomery County,Davis Elementary School,Public Charter School - Converted,1.024300e+10,True,3605 Rosa Parks Dr,Montgomery,AL,36105-2099,Montgomery,2022.0,1/1/01,Montgomery County,51.0,True,Hispanic/Latino
507072,53,Perry County,0,Perry County,Math,ALL,All Gender,All Race,Hispanic/Latino,All SubPopulation,*,*,*,*,*,*,*,*,Perry County,Perry County,Central Office,1.026700e+05,NaN,200 Monroe St,Marion,AL,36756,Perry,2022.0,NaN,Perry County,NaN,True,Hispanic/Latino
562734,60,Sumter County,0,Sumter County,Math,ALL,All Gender,All Race,Hispanic/Latino,All SubPopulation,*,*,*,*,*,*,*,*,Sumter County,Sumter County,Central Office,1.030900e+05,NaN,Hwy 28 To Country Club Dr,Livingston,AL,35470,Sumter,2022.0,NaN,Sumter County,NaN,True,Hispanic/Latino
668350,114,Birmingham City,0,Birmingham City,Math,ALL,All Gender,All Race,Hispanic/Latino,All SubPopulation,0.00,0.63,8.14,56.24,35.62,6.69,1.45,96.87,Birmingham City,Birmingham City,Central Office,1.003900e+05,NaN,2015 Park Place North,Birmingham,AL,35203-0007,Jefferson,2022.0,1/1/01,Birmingham City,NaN,True,Hispanic/Latino
810322,157,Homewood City,0,Homewood City,Math,ALL,All Gender,All Race,Hispanic/Latino,All SubPopulation,0.00,0.00,31.84,20.41,47.76,25.31,6.53,99.71,Homewood City,Homewood City,Central Office,1.017600e+05,NaN,450 Dale Avenue,Homewood,AL,35209-3401,Jefferson,2022.0,NaN,Homewood City,NaN,True,Hispanic/Latino
1016254,800,Acceleration Day and Evening Academy,10,Acceleration Day and Evening Academy,Math,ALL,All Gender,All Race,Hispanic/Latino,All SubPopulation,*,*,*,*,*,*,*,*,Acceleration Day and Evening Academy,Acceleration Day and Evening Academy,Public Charter School Startup,1.001970e+10,True,3725 AIRPORT BLVDSuite 161,MOBILE,AL,36608,Mobile,2022.0,6/1/17,Mobile County,49.0,True,Hispanic/Latino
1018261,801,University Charter School,10,University Charter School,Math,ALL,All Gender,All Race,Hispanic/Latino,All SubPopulation,*,*,*,*,*,*,*,*,University Charter School,University Charter School,Public Charter School Startup,1.001990e+10,True,108 North Street,Livingston,AL,35470,Sumter,2022.0,6/19/18,Sumter County,60.0,True,Hispanic/Latino


In [162]:
df_equity = pd.concat([df_subpops, df_black, df_hispanic])

In [163]:
df_merged_assessment_2022.loc[(df_merged_assessment_2022['school_code']==803) & 
                             (df_merged_assessment_2022['gender']=='All Gender')& 
                             (df_merged_assessment_2022['grade']=='ALL')
                              #& 
                             # (df_merged_assessment_2022['ethnicity']=='All Ethnicity') & 
                             # (df_merged_assessment_2022['race']=='Hispanic/Latino')
                             ]

,system_code,system_name_x,school_code,school_name_x,subject,grade,gender,race,ethnicity,subpopulation,first_year_el_exemption_percent,alternate_assessed_percent,percent_proficient,percent_level1,percent_level2,percent_level3,percent_level4,participation_rate,system_name_y,school_name_y,site_type,nces_id,is_charter,street,city,state,zip_5,county,file_year,opened_date,updated_district,updated_district_code,is_charter_district
695536,114,Birmingham City,803,Richard Arrington Elementary,Math,ALL,All Gender,All Race,All Ethnicity,All SubPopulation,0.00,0.00,2.25,72.47,25.28,1.69,0.56,98.46,Birmingham City,Richard Arrington Elementary,Public Regular School,1.003900e+10,False,2101 Jefferson Avenue SW,Birmingham,AL,35211,Jefferson,2022.0,7/1/19,Birmingham City,NaN,True
695537,114,Birmingham City,803,Richard Arrington Elementary,Math,ALL,All Gender,All Race,Hispanic/Latino,All SubPopulation,0.00,0.00,10.53,57.89,31.58,5.26,5.26,95.00,Birmingham City,Richard Arrington Elementary,Public Regular School,1.003900e+10,False,2101 Jefferson Avenue SW,Birmingham,AL,35211,Jefferson,2022.0,7/1/19,Birmingham City,NaN,True
695538,114,Birmingham City,803,Richard Arrington Elementary,Math,ALL,All Gender,All Race,Other Ethnicity,All SubPopulation,0.00,0.00,1.26,74.21,24.53,1.26,0.00,98.89,Birmingham City,Richard Arrington Elementary,Public Regular School,1.003900e+10,False,2101 Jefferson Avenue SW,Birmingham,AL,35211,Jefferson,2022.0,7/1/19,Birmingham City,NaN,True
695539,114,Birmingham City,803,Richard Arrington Elementary,Math,ALL,All Gender,Black or African American,All Ethnicity,All SubPopulation,0.00,0.00,1.26,74.21,24.53,1.26,0.00,99.38,Birmingham City,Richard Arrington Elementary,Public Regular School,1.003900e+10,False,2101 Jefferson Avenue SW,Birmingham,AL,35211,Jefferson,2022.0,7/1/19,Birmingham City,NaN,True
695540,114,Birmingham City,803,Richard Arrington Elementary,Math,ALL,All Gender,Black or African American,Other Ethnicity,All SubPopulation,0.00,0.00,1.27,74.52,24.20,1.27,0.00,99.37,Birmingham City,Richard Arrington Elementary,Public Regular School,1.003900e+10,False,2101 Jefferson Avenue SW,Birmingham,AL,35211,Jefferson,2022.0,7/1/19,Birmingham City,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1722753,114,Birmingham City,803,Richard Arrington Elementary,ELA,ALL,All Gender,Two or more races,Other Ethnicity,Economically Disadvantaged,*,*,*,*,*,*,*,*,Birmingham City,Richard Arrington Elementary,Public Regular School,1.003900e+10,False,2101 Jefferson Avenue SW,Birmingham,AL,35211,Jefferson,2022.0,7/1/19,Birmingham City,NaN,True
1722881,114,Birmingham City,803,Richard Arrington Elementary,ELA,ALL,All Gender,Black or African American,Other Ethnicity,Foster,*,*,*,*,*,*,*,*,Birmingham City,Richard Arrington Elementary,Public Regular School,1.003900e+10,False,2101 Jefferson Avenue SW,Birmingham,AL,35211,Jefferson,2022.0,7/1/19,Birmingham City,NaN,True
1722882,114,Birmingham City,803,Richard Arrington Elementary,ELA,ALL,All Gender,All Race,All Ethnicity,Foster,*,*,*,*,*,*,*,*,Birmingham City,Richard Arrington Elementary,Public Regular School,1.003900e+10,False,2101 Jefferson Avenue SW,Birmingham,AL,35211,Jefferson,2022.0,7/1/19,Birmingham City,NaN,True
1722883,114,Birmingham City,803,Richard Arrington Elementary,ELA,ALL,All Gender,All Race,Other Ethnicity,Foster,*,*,*,*,*,*,*,*,Birmingham City,Richard Arrington Elementary,Public Regular School,1.003900e+10,False,2101 Jefferson Avenue SW,Birmingham,AL,35211,Jefferson,2022.0,7/1/19,Birmingham City,NaN,True


In [164]:
df_merged_assessment = df_merged_assessment.replace('*',np.NaN).replace('~',np.NaN).drop_duplicates().loc[df_merged_assessment['file_year']>2016]
df_merged_assessment.head()

,system_name,school_name,subject,grade,tested,proficient,participation_rate,proficient_rate,file_source,file_year,system_code,school_code,site_type,nces_id,is_charter,street,city,state,zip_5,county,opened_date
0,Alabama State Department of Education,Alabama State Department of Education,Math,ALL,378650.0,103106.39499999999,98.35,27.23,df_assessment_2022,2022,999.0,0.0,Central Office,NaN,NaN,"50. N. Ripley Street, Gordon Persons Building",Montgomery,AL,36104,Montgomery,1/1/01
1,Alabama State Department of Education,Alabama State Department of Education,Math,ALL,378650.0,103106.39499999999,98.35,27.23,df_assessment_2022,2022,999.0,1.0,State Board District,NaN,NaN,3071 Teal Court,Mobile,AL,36695,Baldwin,1/1/00
2,Autauga County,Autauga County,Math,ALL,4738.0,1413.3454000000002,99.04,29.83,df_assessment_2022,2022,1.0,0.0,Central Office,1.002400e+05,NaN,153 W 4th St,Prattville,AL,36067-3011,Autauga,NaN
3,Autauga County,Autauga County,Math,ALL,4738.0,1413.3454000000002,99.04,29.83,df_assessment_2022,2022,1.0,1.0,Public Board of Education,NaN,NaN,153 W 4th St,Prattville,AL,36067-3011,Autauga,NaN
4,Autauga County,Autaugaville School,Math,ALL,134.0,7.9998,99.18,5.97,df_assessment_2022,2022,1.0,15.0,Public Regular School,1.002400e+10,False,2708 Dutch Bend St,Autaugaville,AL,36003,Autauga,8/1/99


In [165]:
df_all_state = df_merged_assessment.loc[df_merged_assessment['system_code']==999]
df_all_state

,system_name,school_name,subject,grade,tested,proficient,participation_rate,proficient_rate,file_source,file_year,system_code,school_code,site_type,nces_id,is_charter,street,city,state,zip_5,county,opened_date
0,Alabama State Department of Education,Alabama State Department of Education,Math,ALL,378650.0,103106.39499999999,98.35,27.23,df_assessment_2022,2022,999.0,0.0,Central Office,NaN,NaN,"50. N. Ripley Street, Gordon Persons Building",Montgomery,AL,36104,Montgomery,1/1/01
1,Alabama State Department of Education,Alabama State Department of Education,Math,ALL,378650.0,103106.39499999999,98.35,27.23,df_assessment_2022,2022,999.0,1.0,State Board District,NaN,NaN,3071 Teal Court,Mobile,AL,36695,Baldwin,1/1/00
1698,Alabama State Department of Education,Alabama State Department of Education,ELA,ALL,376925.0,178210.13999999998,98.04,47.28,df_assessment_2022,2022,999.0,0.0,Central Office,NaN,NaN,"50. N. Ripley Street, Gordon Persons Building",Montgomery,AL,36104,Montgomery,1/1/01
1699,Alabama State Department of Education,Alabama State Department of Education,ELA,ALL,376925.0,178210.13999999998,98.04,47.28,df_assessment_2022,2022,999.0,1.0,State Board District,NaN,NaN,3071 Teal Court,Mobile,AL,36695,Baldwin,1/1/00
3399,Alabama State Department of Education,Alabama State Department of Education,Math,All Grades,364253.00,80087,93.70,21.99,df_assessment_2021,2021,999.0,0.0,Central Office,NaN,NaN,"50. N. Ripley Street, Gordon Persons Building",Montgomery,AL,36104,Montgomery,1/1/01
3400,Alabama State Department of Education,Alabama State Department of Education,Math,All Grades,364253.00,80087,93.70,21.99,df_assessment_2021,2021,999.0,1.0,State Board District,NaN,NaN,3071 Teal Court,Mobile,AL,36695,Baldwin,1/1/00
5011,Alabama State Department of Education,Alabama State Department of Education,ELA,All Grades,363985.00,164921,93.62,45.39,df_assessment_2021,2021,999.0,0.0,Central Office,NaN,NaN,"50. N. Ripley Street, Gordon Persons Building",Montgomery,AL,36104,Montgomery,1/1/01
5012,Alabama State Department of Education,Alabama State Department of Education,ELA,All Grades,363985.00,164921,93.62,45.39,df_assessment_2021,2021,999.0,1.0,State Board District,NaN,NaN,3071 Teal Court,Mobile,AL,36695,Baldwin,1/1/00
6623,Alabama State Department of Education,Alabama State Department of Education,Math,All Grades,388991.00,180956,NaN,46.52,df_assessment_2019,2019,999.0,0.0,Central Office,NaN,NaN,"50. N. Ripley Street, Gordon Persons Building",Montgomery,AL,36104,Montgomery,1/1/01
6624,Alabama State Department of Education,Alabama State Department of Education,Math,All Grades,388991.00,180956,NaN,46.52,df_assessment_2019,2019,999.0,1.0,State Board District,NaN,NaN,3071 Teal Court,Mobile,AL,36695,Baldwin,1/1/00


In [166]:
df_merged_assessment['tested_derived'] = df_merged_assessment['proficient'].astype('float') / (df_merged_assessment['proficient_rate'].astype('float')/100)
df_merged_assessment['tested_derived'] = np.where(df_merged_assessment['tested'].isnull(), df_merged_assessment['tested_derived'], df_merged_assessment['tested'])

In [167]:
df_merged_assessment['subject'] = df_merged_assessment['subject'].replace('Reading','ELA')
df_merged_assessment['subject'].unique()

array(['Math', 'ELA'], dtype=object)

In [168]:
df_merged_assessment['grade'] = np.where(df_merged_assessment['grade']=='Grade 03', '03',
         np.where(df_merged_assessment['grade']=='Grade 04', '04',
         np.where(df_merged_assessment['grade']=='Grade 05', '05',
         np.where(df_merged_assessment['grade']=='Grade 05', '05',
         np.where(df_merged_assessment['grade']=='Grade 06', '06',
         np.where(df_merged_assessment['grade']=='Grade 07', '07',
         np.where(df_merged_assessment['grade']=='Grade 08', '08',
         np.where(df_merged_assessment['grade']=='All Grades', 'ALL',
                  df_merged_assessment['grade']
                 ))))))))

In [169]:
df_merged_assessment['is_charter'].unique()

array([nan, False, True], dtype=object)

In [170]:
df_merged_assessment['grade'].unique()

array(['ALL'], dtype=object)

In [171]:
# ## Manually adding in some LEAD Academy n-values 
# df_merged_assessment.at[6716, 'tested']=64 ## 271-207 
# df_merged_assessment.at[6716, 'tested_derived']=64 ## 271-207 
# df_merged_assessment.at[6716, 'proficient']=3 ## 64 * .0469 OR 9 - 6 ; both are derived 

# df_merged_assessment.at[13572, 'tested']=64 ## 271-207 
# df_merged_assessment.at[13572, 'tested_derived']=64 ## 271-207 
# df_merged_assessment.at[13572, 'proficient']=20 ## 64 * .3125 OR 67-47 ; both are derived 

df_merged_assessment.loc[(df_merged_assessment['system_name']=='i3 Academy') & (df_merged_assessment['grade']=='ALL')]

,system_name,school_name,subject,grade,tested,proficient,participation_rate,proficient_rate,file_source,file_year,system_code,school_code,site_type,nces_id,is_charter,street,city,state,zip_5,county,opened_date,tested_derived
1680,i3 Academy,i3 Academy,Math,ALL,308.0,25.0096,98.09,8.12,df_assessment_2022,2022,805.0,0.0,Central Office,1.002040e+05,NaN,55th Place 1st Avenue South,Birmingham,AL,35212,Jefferson,7/1/20,308.0
1681,i3 Academy,i3 Academy,Math,ALL,308.0,25.0096,98.09,8.12,df_assessment_2022,2022,805.0,1.0,Public Board of Education,1.002040e+10,NaN,55th Place 1st Avenue South,Birmingham,AL,35212-,Jefferson,7/1/20,308.0
1682,i3 Academy,i3 Academy - Elementary School,Math,ALL,210.0,16.002000000000002,97.67,7.62,df_assessment_2022,2022,805.0,10.0,Public Charter School Startup,1.002040e+10,True,55th Place 1st Avenue South,Birmingham,AL,35212,Jefferson,7/1/20,210.0
1683,i3 Academy,i3 Academy - Middle School,Math,ALL,98.0,8.9964,98.99,9.18,df_assessment_2022,2022,805.0,15.0,Public Charter School Startup,1.002040e+10,True,7901 1st Avenue North,Birmingham,AL,35206-,Jefferson,7/1/21,98.0
3378,i3 Academy,i3 Academy,ELA,ALL,305.0,85.278,97.13,27.96,df_assessment_2022,2022,805.0,0.0,Central Office,1.002040e+05,NaN,55th Place 1st Avenue South,Birmingham,AL,35212,Jefferson,7/1/20,305.0
3379,i3 Academy,i3 Academy,ELA,ALL,305.0,85.278,97.13,27.96,df_assessment_2022,2022,805.0,1.0,Public Board of Education,1.002040e+10,NaN,55th Place 1st Avenue South,Birmingham,AL,35212-,Jefferson,7/1/20,305.0
3380,i3 Academy,i3 Academy - Elementary School,ELA,ALL,207.0,54.9999,96.28,26.57,df_assessment_2022,2022,805.0,10.0,Public Charter School Startup,1.002040e+10,True,55th Place 1st Avenue South,Birmingham,AL,35212,Jefferson,7/1/20,207.0
3381,i3 Academy,i3 Academy - Middle School,ELA,ALL,98.0,30.311400000000003,98.99,30.93,df_assessment_2022,2022,805.0,15.0,Public Charter School Startup,1.002040e+10,True,7901 1st Avenue North,Birmingham,AL,35206-,Jefferson,7/1/21,98.0
3539,i3 Academy,i3 Academy - Elementary School,Math,ALL,219.0159850529375,8.01598505293751,NaN,3.6599999999999966,df_assessment_2021,2021,805.0,10.0,Public Charter School Startup,1.002040e+10,True,55th Place 1st Avenue South,Birmingham,AL,35212,Jefferson,7/1/20,219.0159850529375
5151,i3 Academy,i3 Academy - Elementary School,ELA,ALL,217.97631862217435,56,NaN,25.69,df_assessment_2021,2021,805.0,10.0,Public Charter School Startup,1.002040e+10,True,55th Place 1st Avenue South,Birmingham,AL,35212,Jefferson,7/1/20,217.97631862217435


In [172]:
df_charter_codes = df_merged_assessment.loc[(df_merged_assessment['file_year']==2022) &
                        (df_merged_assessment['is_charter']==True) &
                        (df_merged_assessment['grade']=='ALL')][['system_code']].drop_duplicates()

df_charter_central_office = df_merged_assessment.merge(df_charter_codes, left_on='system_code', right_on='system_code', how='inner')

df_charter_schools_2022 = df_charter_central_office.loc[(df_charter_central_office['grade']=='ALL') & 
                                                        (df_charter_central_office['school_code']!=1)&
                                                        (df_charter_central_office['file_year']==2022)]

df_charter_count_2022 = pd.DataFrame(df_charter_schools_2022.groupby(['system_code','subject'])['school_name'].nunique()).reset_index()
df_charter_tested_2022 = df_charter_central_office.loc[(df_charter_central_office['school_code']==1) & 
                                                               (df_charter_central_office['grade']=='ALL') & 
                                                               (df_charter_central_office['file_year']==2022)][['system_code','subject','tested_derived']]

df_charter_central = (
    df_merged_assessment
    .merge(df_charter_count_2022, left_on=['system_code','subject'],right_on=['system_code','subject'], how='inner')
    .merge(df_charter_tested_2022, left_on=['system_code','subject'],right_on=['system_code','subject'], how='inner')
)

df_charter_central.loc[(df_charter_central['grade']=='ALL') 
                       & (df_charter_central['file_year']==2021) 
                       & (df_charter_central['school_code']!=1) 
                       & (df_charter_central['school_code']!=0)]
# df_charter_schools_2022

,system_name,school_name_x,subject,grade,tested,proficient,participation_rate,proficient_rate,file_source,file_year,system_code,school_code,site_type,nces_id,is_charter,street,city,state,zip_5,county,opened_date,tested_derived_x,school_name_y,tested_derived_y
53,Montgomery County,Baldwin Art And Academics Magnet,Math,ALL,460.00,164,85.03,35.65,df_assessment_2021,2021,51.0,20.0,Public Magnet School,1.024300e+10,False,410 S McDonough St,Montgomery,AL,36104-4226,Montgomery,NaN,460.00,50,13398.0
54,Montgomery County,Bear Exploration Center,Math,ALL,267.0278637770898,60,NaN,22.47,df_assessment_2021,2021,51.0,30.0,Public Magnet School,1.024300e+10,False,2525 Churchill Dr,Montgomery,AL,36111-2899,Montgomery,NaN,267.0278637770898,50,13398.0
55,Montgomery County,Bellingrath Middle School,Math,ALL,284.00,2.9818089944416215,49.04,1.0499999999999954,df_assessment_2021,2021,51.0,50.0,Public Regular School,1.024300e+10,False,3350 S Court St,Montgomery,AL,36105-1699,Montgomery,1/1/01,284.00,50,13398.0
56,Montgomery County,Blount Elementary School,Math,ALL,109.00,35,90.08,32.11,df_assessment_2021,2021,51.0,55.0,Public Regular School,1.024300e+10,False,1650 Ray Thorington Road,Montgomery,AL,36117-8487,Montgomery,8/1/03,109.00,50,13398.0
57,Montgomery County,Booker T Washington Magnet High School,Math,ALL,93.00,31,89.42,33.33,df_assessment_2021,2021,51.0,60.0,Public Magnet School,1.024300e+10,False,3315 Hayneville Road,Montgomery,AL,36108-3902,Montgomery,NaN,93.00,50,13398.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566,Acceleration Day and Evening Academy,Acceleration Day and Evening Academy,ELA,ALL,47.00,2.9985045930356833,52.53,6.380000000000008,df_assessment_2021,2021,800.0,10.0,Public Charter School Startup,1.001970e+10,True,3100 Cottage Hill Rd,Mobile,AL,36606,Mobile,6/1/17,47.00,1,141.0
577,University Charter School,University Charter School,ELA,ALL,258.99280575539564,133,NaN,51.35,df_assessment_2021,2021,801.0,10.0,Public Charter School Startup,1.001990e+10,True,108 North Street,Livingston,AL,35470,Sumter,6/19/18,258.99280575539564,1,295.0
586,LEAD Academy,LEAD Academy - Elementary School,ELA,ALL,172.0011965300628,57,NaN,33.14,df_assessment_2021,2021,802.0,10.0,Public Charter School Startup,1.002030e+10,True,2897 Eastern Blvd,Montgomery,AL,36116-1021,Montgomery,7/1/19,172.0011965300628,3,271.0
591,Legacy Prep,Legacy Prep,ELA,ALL,NaN,NaN,NaN,NaN,df_assessment_2021,2021,803.0,10.0,Public Charter School Startup,1.002010e+10,True,1500 Daniel Payne Drive,Birmingham,AL,35214-,Washington,7/1/19,NaN,1,90.0


## Assessments

In [173]:
df_assessment_school_drop_charter_dupes = df_merged_assessment.loc[(df_merged_assessment['school_code']!=0) & (df_merged_assessment['school_code']!=1) & (df_merged_assessment['school_code']!=1)].drop_duplicates()
df_assessment_school_drop_charter_dupes['is_charter'] = df_assessment_school_drop_charter_dupes['is_charter'].replace(np.NaN,'False')


In [174]:
df_district_charter_mapping = pd.DataFrame({'system_code':
    [800,51,802,802,808,801,803,805,805,810],
    'school_code':
    [10,220,10,15,10,10,10,10,15,10],
    'updated_district':
    ['Mobile County','Montgomery County','Montgomery County','Montgomery County','Perry County','Sumter County','Birmingham City','Birmingham City','Birmingham City','Homewood City'],
    'updated_district_code':[49,51,51,51,53,60,114,114,114,157]
})

df_assessment_school_drop_charter_dupes = df_assessment_school_drop_charter_dupes.merge(df_district_charter_mapping, on=['system_code','school_code'], how='left')
df_assessment_school_drop_charter_dupes['updated_district'].fillna(value=df_assessment_school_drop_charter_dupes['system_name'], inplace=True)
df_assessment_school_drop_charter_dupes['updated_district_code'].fillna(value=df_assessment_school_drop_charter_dupes['system_code'], inplace=True)
df_assessment_school_drop_charter_dupes

,system_name,school_name,subject,grade,tested,proficient,participation_rate,proficient_rate,file_source,file_year,system_code,school_code,site_type,nces_id,is_charter,street,city,state,zip_5,county,opened_date,tested_derived,updated_district,updated_district_code
0,Autauga County,Autaugaville School,Math,ALL,134.0,7.9998,99.18,5.97,df_assessment_2022,2022,1.0,15.0,Public Regular School,1.002400e+10,False,2708 Dutch Bend St,Autaugaville,AL,36003,Autauga,8/1/99,134.0,Autauga County,1.0
1,Autauga County,Billingsley High School,Math,ALL,290.0,37.004,99.37,12.76,df_assessment_2022,2022,1.0,40.0,Public Regular School,1.002400e+10,False,2446 County Road 77,Billingsley,AL,36006,Autauga,NaN,290.0,Autauga County,1.0
2,Autauga County,Marbury High School,Math,ALL,134.0,23.9994,98.00,17.91,df_assessment_2022,2022,1.0,50.0,Public Regular School,1.002400e+10,False,2360 Highway 31 North,Deatsville,AL,36022,Autauga,1/1/01,134.0,Autauga County,1.0
3,Autauga County,Marbury Middle School,Math,ALL,443.0,85.0117,99.12,19.19,df_assessment_2022,2022,1.0,52.0,Public Regular School,1.002400e+10,False,210 County Road 20 East,Marbury,AL,36051,Autauga,10/1/09,443.0,Autauga County,1.0
4,Autauga County,Pine Level Elementary School,Math,ALL,405.0,173.988,100.00,42.96,df_assessment_2022,2022,1.0,55.0,Public Regular School,1.002400e+10,False,2040 Highway 31 N,Deatsville,AL,36022,Autauga,1/1/01,405.0,Autauga County,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12782,Birmingham City,Woodlawn High School-Magnet,ELA,ALL,174.00,20,82.46,11.49,df_assessment_2017,2017,114.0,920.0,Public Magnet School,1.003900e+10,False,5620 First Ave N,Birmingham,AL,35212,Jefferson,1/1/94,174.00,Birmingham City,114.0
12783,Birmingham City,Wylam K-8,ELA,ALL,258.0059639569558,59,NaN,22.87,df_assessment_2017,2017,114.0,930.0,Public Regular School,1.003900e+10,False,701 Erie Street,Birmingham,AL,35224-2655,Jefferson,1/1/94,258.0059639569558,Birmingham City,114.0
12784,Jefferson County,West Jefferson Elementary School,ELA,ALL,71.99696854869268,34,NaN,47.22,df_assessment_2017,2017,37.0,940.0,Public Regular School,1.019200e+10,False,4880 Freewill Dr,Quinton,AL,35130,Jefferson,10/24/00,71.99696854869268,Jefferson County,37.0
12785,Gadsden City,C I T Y Program,ELA,ALL,0.00,NaN,0.00,0.00,df_assessment_2017,2017,144.0,3000.0,Public Alternative School,1.016200e+10,False,435 E Chestnut,Gadsden,AL,35903,Etowah,1/1/94,0.00,Gadsden City,144.0


In [175]:
with pd.ExcelWriter('../data_for_analysis/asmt.xlsx') as writer:  
    df_assessment_school_drop_charter_dupes.to_excel(writer, sheet_name='assessment_all_schools')
    df_all_state.to_excel(writer, sheet_name='assessment_all_state')
    df_equity.to_excel(writer, sheet_name='equity')